In [1]:
import pandas as pd

In [11]:
df1 = pd.read_csv('D:\\Python_work\\exercise_data\\2015年国内主要城市年度数据.csv')
df2 = pd.read_csv('D:\\Python_work\\exercise_data\\2016年国内主要城市年度数据.csv')
df3 = pd.read_csv('D:\\Python_work\\exercise_data\\2017年国内主要城市年度数据.csv')

In [13]:
df1.head()

,地区,年份,国内生产总值,第一产业增加值,第二产业增加值,第三产业增加值,社会商品零售总额,货物进出口总额,年末总人口,在岗职工平均工资,普通高等学校在校学生数,医院、卫生院数,房地产开发投资额
0,北京,2015,23014.59,140.21,4542.64,18331.74,10338.0,319416.16,1345.20,113073,60.36,701,4177.05
1,天津,2015,16538.19,208.82,7704.22,8625.15,5257.3,114282.80,1026.90,81486,51.29,661,1871.55
2,石家庄,2015,5440.60,494.44,2452.40,2493.77,2693.0,12160.29,1028.84,54441,41.98,393,965.13
3,太原,2015,2735.34,37.40,1020.18,1677.77,1540.8,10677.38,367.39,60516,42.14,247,597.83
4,呼和浩特,2015,3090.52,126.23,867.08,2097.21,1353.5,2072.75,238.58,53698,23.52,174,509.05


In [17]:
# 1.纵向合并
combined_df = pd.concat([df1, df2, df3], ignore_index=True)
# 2.处理缺失值：填充为0
combined_df.fillna(0, inplace=True)

In [19]:
# 3. 按年份聚合计算GDP总和
# 4.求国内生产总值    
annual_gdp = combined_df.groupby('年份')['国内生产总值'].sum().reset_index()
annual_gdp.columns = ['Year', 'Total_GDP']

In [21]:
print(annual_gdp)

   Year  Total_GDP
0  2015  277958.37
1  2016  300696.10
2  2017  332562.43


In [33]:
# 5.计算每个城市2015-2017年GDP的年均增长率，并找出增长率最高和最低的五个城市
def calculate_growth_rate(group):
    """
    计算城市2015-2017年GDP的年均增长率（CAGR）
    :param group: 按城市分组后的DataFrame
    :return: 年均增长率（浮点数），若数据不足或异常则返回0
    """
    # 提取2015年和2017年的GDP数据
    gdp_2015 = group[group['年份'] == 2015]['国内生产总值'].values
    gdp_2017 = group[group['年份'] == 2017]['国内生产总值'].values
    
    # 检查数据是否存在
    if len(gdp_2015) == 0 or len(gdp_2017) == 0:
        return 0  # 缺少2015或2017年数据
    
    gdp_2015 = gdp_2015[0]
    gdp_2017 = gdp_2017[0]
    
    # 检查数据有效性
    if gdp_2015 <= 0:  # GDP必须为正数
        return 0
    
    # 计算CAGR（2015→2017，2年间隔）
    growth_rate = (gdp_2017 / gdp_2015) ** (1/2) - 1
    return growth_rate

# 按城市分组计算增长率
city_growth = combined_df.groupby('地区').apply(calculate_growth_rate).reset_index()
city_growth.columns = ['地区', 'GDP年均增长率']
 
# 找出增长率最高和最低的5个城市
top5_growth = city_growth.nlargest(5, 'GDP年均增长率')
bottom5_growth = city_growth.nsmallest(5, 'GDP年均增长率')
 
print("GDP年均增长率最高的5个地区:")
print(top5_growth)
print("\nGDP年均增长率最低的5个地区:")
print(bottom5_growth)

GDP年均增长率最高的5个地区:
    地区  GDP年均增长率
28  西安  0.134742
16  成都  0.133983
24  深圳  0.133550
8   合肥  0.128893
17  拉萨  0.127888

GDP年均增长率最低的5个地区:
      地区  GDP年均增长率
21    沈阳 -0.101958
9   呼和浩特 -0.057776
11    大连 -0.024070
1   乌鲁木齐  0.021091
10   哈尔滨  0.051187


C:\Users\和彦汝\AppData\Local\Temp\ipykernel_30988\2149415442.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  city_growth = combined_df.groupby('地区').apply(calculate_growth_rate).reset_index()


In [25]:
# 6.对医院、卫生院数进行归一化处理（Min-Max标准化），并按年份比较各城市医疗资源的变化
# 提取医院数数据（假设列名为'医院、卫生院数'）
hospital_data = combined_df[['地区', '年份', '医院、卫生院数']].copy()
 
# Min-Max归一化
def min_max_normalize(group):
    min_val = group['医院、卫生院数'].min()
    max_val = group['医院、卫生院数'].max()
    group['归一化医院数'] = (group['医院、卫生院数'] - min_val) / (max_val - min_val + 1e-6)  # 避免除以0
    return group
 
normalized_hospital = hospital_data.groupby('地区').apply(min_max_normalize)
 
# 按年份比较医疗资源变化（计算每年平均值）
annual_hospital = normalized_hospital.groupby('年份')['归一化医院数'].mean().reset_index()
print("\n各年份医疗资源归一化平均值:")
print(annual_hospital)


各年份医疗资源归一化平均值:
     年份    归一化医院数
0  2015  0.768986
1  2016  0.941834
2  2017  0.097012


C:\Users\和彦汝\AppData\Local\Temp\ipykernel_30988\3306077608.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  normalized_hospital = hospital_data.groupby('地区').apply(min_max_normalize)


In [29]:
# 7.提取北京、上海、广州、深圳四个城市2015-2017的GDP和社会商品零售总额数据，用新的csv呈现
# 一线城市列表
tier1_cities = ['北京', '上海', '广州', '深圳']
 
# 提取相关数据（假设有'社会商品零售总额'列）
required_columns = ['地区', '年份', '国内生产总值', '社会商品零售总额']
tier1_data = combined_df[combined_df['地区'].isin(tier1_cities)][required_columns]
 
# 保存到新CSV
tier1_data.to_csv('一线城市2015-2017经济数据.csv', index=False, encoding='utf_8_sig')
print("\n已保存: 一线城市2015-2017经济数据.csv")



已保存: 一线城市2015-2017经济数据.csv
